In [ ]:
import os
import cv2
import numpy as np
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from qiskit import Aer, QuantumCircuit, transpile, assemble
from qiskit.circuit import Parameter

male_dir = "E:\python files\Male"
female_dir = "E:\python files\Female"

# Preprocessing function to resize and preprocess images
def preprocess(img):
    img = cv2.resize(img, (100, 100))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = img.flatten()
    return img

# Load male and female images into arrays and preprocess them
male_images = []
for img_file in os.listdir(male_dir):
    img = cv2.imread(os.path.join(male_dir, img_file))
    img = preprocess(img)
    male_images.append(img)

female_images = []
for img_file in os.listdir(female_dir):
    img = cv2.imread(os.path.join(female_dir, img_file))
    img = preprocess(img)
    female_images.append(img)

# Combine male and female images into a single dataset and labels array
X = np.concatenate((male_images, female_images))
y = np.concatenate((np.ones(len(male_images)), np.zeros(len(female_images))))

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Encode labels as integers
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)
# Quantum Convolutional Layer
def quantum_convolution_layer(circuit, input_qubits, weight_qubits, theta, num_qubits_conv_layer):
    circuit.h(input_qubits)
    circuit.cx(input_qubits, weight_qubits)
    circuit.ry(theta, weight_qubits)
    circuit.cx(input_qubits, weight_qubits)
    circuit.h(input_qubits)

# Quantum Convolutional Neural Network with Amplitude Encoding
def quantum_convolutional_neural_network(data, weights):
    # Pad the data array to make its length a power of 2
    n_pad = int(np.ceil(np.log2(len(data))))
    data_padded = np.pad(data, (0, 2**n_pad - len(data)), 'constant', constant_values=0)

    # Normalize the data to represent the amplitudes of the quantum state
    amplitudes = data_padded / np.linalg.norm(data_padded)

    # Calculate the number of qubits needed for the quantum circuit
    num_qubits = int(np.log2(len(amplitudes)))

    circuit = QuantumCircuit(num_qubits + len(weights), len(weights))
    circuit.initialize(amplitudes, range(num_qubits))
    circuit.barrier()

    for i, weight in enumerate(weights):
        # Get the number of qubits required by the convolutional layer
        num_qubits_conv_layer = len(weight)
        
        # Apply the quantum convolutional layer to the appropriate qubits
        quantum_convolution_layer(circuit, range(num_qubits), range(num_qubits, num_qubits + num_qubits_conv_layer), weight, num_qubits_conv_layer)
        circuit.barrier()

    return circuit

X_train_quantum = [np.arccos(np.sqrt(x/sum(x))) for x in X_train]
X_test_quantum = [np.arccos(np.sqrt(x/sum(x))) for x in X_test]

# Initialize random weights for the quantum convolutional layer
num_weights = 4  # You can set this to any desired number
weights = [Parameter('w' + str(i)) for i in range(num_weights)]

# Create a quantum convolutional neural network circuit
qcnn_circuit = quantum_convolutional_neural_network(X_train_quantum[0], weights)

# Use Aer's qasm_simulator
backend = Aer.get_backend('qasm_simulator')

# Execute the quantum circuit on the simulator
tqc = transpile(qcnn_circuit, backend)
qobj = assemble(tqc)
result = backend.run(qobj).result()
counts = result.get_counts(qcnn_circuit)

# Train SVM model on quantum circuit output for training data
X_train_qcnn = []
for data in X_train_quantum:
    tqc = transpile(quantum_convolutional_neural_network(data, weights), backend)
    qobj = assemble(tqc)
    result = backend.run(qobj).result()
    counts = result.get_counts()
    X_train_qcnn.append([counts.get('0', 0), counts.get('1', 0)])

clf = svm.SVC(kernel='linear')
clf.fit(X_train_qcnn, y_train)

# Test SVM model on quantum circuit output for testing data and evaluate accuracy
X_test_qcnn = []
for data in X_test_quantum:
    tqc = transpile(quantum_convolutional_neural_network(data, weights), backend)
    qobj = assemble(tqc)
    result = backend.run(qobj).result()
    counts = result.get_counts()
    X_test_qcnn.append([counts.get('0', 0), counts.get('1', 0)])

y_pred = clf.predict(X_test_qcnn)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
# Python code to demonstrate naive method
# to compute factorial
n = 23
fact = 1

for i in range(1, n+1):
	fact = fact * i

print("The factorial of 23 is : ", end="")
print(fact)